In [ ]:
#hide
from nbdev import *
%nbdev_default_export release

Cells will be exported to nbdev.release,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Create release notes

> Release notes auto-generated from GitHub issues

In [ ]:
#export
from datetime import datetime
from textwrap import fill
from urllib.request import Request,urlopen
from urllib.error import HTTPError
from concurrent.futures import ProcessPoolExecutor

In [ ]:
#hide
from nbdev.showdoc import show_doc

In [ ]:
#export
GH_HOST = "https://api.github.com"

In [ ]:
#export
def _issue_txt(issue):
    res = f'- {issue["title"].strip()} ([#{issue["number"]}]({issue["url"]}))\n'
    body = issue['body']
    if not body: return res
    return res + fill(body.strip(), initial_indent="  - ", subsequent_indent="    ") + "\n"

def _issues_txt(iss, label):
    if not iss: return ''
    res = f"### {label}\n\n"
    return res + '\n'.join(map(_issue_txt, iss))

def _config(cfg_name="settings.ini"):
    cfg_path = Path()
    while cfg_path != cfg_path.parent and not (cfg_path/cfg_name).exists(): cfg_path = cfg_path.parent
    config_file = cfg_path/cfg_name
    assert config_file.exists(), f"Couldn't find {cfg_name}"
    config = ConfigParser()
    config.read(config_file)
    return config['DEFAULT'],cfg_path

In [ ]:
#export
class FastRelease:
    def __init__(self, owner=None, repo=None, token=None, **groups):
        "Create CHANGELOG.md from GitHub issues"
        if not groups: groups = dict(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")
        cfg,cfg_path = _config()
        os.chdir(cfg_path)
        if not owner: owner = cfg['user']
        if not repo:  repo  = cfg['lib_name']
        if not token:
            assert Path('token').exists, "Failed to find token"
            token = Path('token').read_text().strip()
        self.owner,self.repo,self.token,self.groups = owner,repo,token,groups
        self.headers = { 'Authorization' : f'token {token}' }
        self.repo_url = f"{GH_HOST}/repos/{owner}/{repo}"

    def _get(self, path, complete=False):
        if not complete: path = f"{self.repo_url}/{path}"
        with urlopen(Request(path, headers=self.headers)) as res: return json.loads(res.read())

    def _tag_date(self, tag):
        try: tag_d = self._get(f"git/ref/tags/{tag}")
        except HTTPError: raise Exception(f"Failed to find tag {tag}")
        commit_d = self._get(tag_d["object"]["url"], True)
        self.commit_date = commit_d["committer"]["date"].replace('Z', '+00:00')
        return self.commit_date

    def _issues(self, label):
        return self._get(f"issues?state=closed&sort=created&filter=all&since={self.commit_date}&labels={label}")
    
    def _issue_groups(self):
        with ProcessPoolExecutor() as ex: return ex.map(self._issues, self.groups.keys())
        
    def _latest_release(self): return self._get("releases/latest")["tag_name"]

    def changelog(self, debug=False):
        "Create the CHANGELOG.md file, or return the proposed text if `debug` is `True`"
        fn = 'CHANGELOG.md'
        shutil.copy(fn, fn+".bak")
        txt = Path(fn).read_text()
        marker = '<!-- do not remove -->\n'
        ver = self._latest_release()
        self._tag_date(ver)
        res = f"## {ver}\n"
        issues = self._issue_groups()
        res += '\n'.join(_issues_txt(*o) for o in zip(issues, self.groups.values()))
        res = txt.replace(marker, marker+res)
        if debug: return res
        else: Path(fn).write_text(res)

To create a markdown changelog, first create a `FastRelease` object, passing a mapping from GitHub labels to markdown titles. Put your github token in a file named `token` at the root of your repo.

In [ ]:
show_doc(FastRelease.changelog)

<h4 id="FastRelease.changelog" class="doc_header"><code>FastRelease.changelog</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>FastRelease.changelog</code>(**`debug`**=*`False`*)

Create the CHANGELOG.md file, or return the proposed text if `debug` is `True`

In [ ]:
# rel = FastRelease(breaking="Breaking Changes", enhancement="New Features", bug="Bugs Squashed")
# print(rel.changelog(debug=True))

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_export.ipynb.
Converted 01_sync.ipynb.
Converted 02_showdoc.ipynb.
Converted 03_export2html.ipynb.
Converted 04_test.ipynb.
Converted 05_merge.ipynb.
Converted 05a_conda.ipynb.
Converted 06_cli.ipynb.
Converted 07_clean.ipynb.
Converted 10_release.ipynb.
Converted 99_search.ipynb.
Converted index.ipynb.
Converted magic_flags.ipynb.
Converted nbdev_callbacks.ipynb.
Converted tutorial.ipynb.
